In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead, row_number, min,max, first
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

import json
params = json.load(open('./mock/parametros.json'))

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# import mysql.connector as database
# import os

# connection = database.connect(
#     host="host.docker.internal",
#     port=3306,
#     user="root",
#     password="secret"
# )


# cursor = connection.cursor()
# cursor.execute(query)

In [3]:
number = 1

In [124]:
fps = 30
vel_media = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}
n_vel_media = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}

tempo_medio = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}
n_tempo_medio = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['velocidade'])
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean("velocidade"))
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    mean_dict = {row["rodovia"]: row["avg(velocidade)"] for row in mean_rows}

    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("velocidade"))
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row["rodovia"]: row["avg(velocidade)"] for row in length_rows}
    n_vel_media = {key: n_vel_media[key]+length_dict[key] for key in list(n_vel_media.keys())}
    vel_media = {key: atualiza_media(vel_media[key], n_vel_media[key], mean_dict[key], length_dict[key]) for key in list(vel_media.keys())}
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter((col("tempo_inicio").isNotNull()) & (col("tempo_final").isNotNull()))
    batch = batch.withColumn("tempo_cruzamento", col("tempo_final") - col("tempo_inicio"))
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean("tempo_cruzamento"))
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    mean_dict = {row["rodovia"]: row["avg(tempo_cruzamento)"] for row in mean_rows}

    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("tempo_cruzamento"))
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row["rodovia"]: row["avg(tempo_cruzamento)"] for row in length_rows}
    n_tempo_medio = {key: n_tempo_medio[key]+length_dict[key] for key in list(n_tempo_medio.keys())}
    tempo_medio = {key: atualiza_media(tempo_medio[key], n_tempo_medio[key], mean_dict[key], length_dict[key]) for key in list(tempo_medio.keys())}
    pass

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    time0 = time.time()
    DadosCarros = DadosCarros.drop('aplicaMulta')
    time1 = time.time()
    print('Drop tempo: ',time1-time0)
    time0 = time.time()
    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('faixa_nova'),
                                   F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')

    time1 = time.time()
    print('Join tempo: ',time1-time0)
    time0 = time.time()
    
    CarrosSumidos = DadosCarros.join(DadosNovos, on="placa", how='left_anti')
    processa_tempo_cruzamento(CarrosSumidos)
    data_joined = data_joined.withColumn("tempo_inicio", when(col("tempo_inicio").isNull(), col("tempo_da_simulacao_novo")).otherwise(col("tempo_inicio")))
    data_joined = data_joined.withColumn("tempo_em_curso", col("tempo_da_simulacao_novo") - col("tempo_inicio"))
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("troca_de_faixa", col("faixa_nova") != col("faixa"))
    data_joined = data_joined.withColumn("faixa", coalesce(col("faixa_nova"), col("faixa")))
    data_joined = data_joined.drop("rodovia_nova", "faixa_nova")

    time1 = time.time()
    print('Cruzamento tempo: ',time1-time0)
    time0 = time.time()
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    processa_velocidade_media(data_joined)
    
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    time1 = time.time()
    print('With tempo: ',time1-time0)
    time0 = time.time()

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    Aceleracoes_Maximas = Parametros.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
    Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")
    
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")
    data_joined = data_joined.join(Aceleracoes_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))
    data_joined = data_joined.withColumn("acima_acel", F.when(F.abs(data_joined["aceleracao"]) > F.abs(fps/data_joined["AceleracaoMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )



    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")
    data_joined = data_joined.drop("AceleracaoMaxima")

    time1 = time.time()
    print('Acima tempo: ',time1-time0)
    time0 = time.time()

    window_spec = Window.partitionBy("rodovia", "faixa").orderBy('posicao')
    
    # Use lag function with the window specification
    lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec)
    lead_column = lead(col("posicao_prevista")).over(window_spec) - col("posicao_prevista")

    
    # Add the lag column to the DataFrame
    data_joined = data_joined.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec)) & (col("faixa") == lag(col("faixa")).over(window_spec)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec)) & (col("faixa") == lead(col("faixa")).over(window_spec))), 1).otherwise(0))

    time1 = time.time()
    print('Window tempo: ',time1-time0)
    time0 = time.time()
    
    # Show the result
    return data_joined
    

In [5]:
df_multas = ss.createDataFrame([], "placa: string, tempo_da_simulacao: int, multa_numero : int")
def aplica_multa(df):
    global df_multas
    df = df.filter(F.col('aplicaMulta') == True)
    df = df.select(F.col('placa'),F.col('tempo_da_simulacao'))
    df2 = df_multas.groupBy('placa').agg(F.count('placa').alias('multa_numero'))
    df2 = df2.withColumn('multa_numero',F.col('multa_numero')+1)  
    df2 = df2.join(df,['placa'],how='right')
    df2 = df2.withColumn('multa_numero', F.when(F.isnull('multa_numero'), 1).otherwise(F.col('multa_numero')))
    df_multas = df2.union(df_multas)

In [6]:
df_perigosa = ss.createDataFrame([], "placa: string, tempo_da_simulacao: int, perigosa_numero : int, tipo: string")
# def perigosa_acel(df,df_perigosa):
#     df = df.filter(F.col('acima_acel') == 1).select(F.col('placa'),F.col('tempo_da_simulacao'))
#     window = Window.partitionBy("placa")
#     df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
#     df2 = df2.join(df,['placa'],how='right')
#     df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit('acima_acel')).union(df_perigosa) 

# def perigosa_faixa(df,df_perigosa):
#     df = df.filter(F.col('troca_de_faixa') == True).select(F.col('placa'),F.col('tempo_da_simulacao'))
#     window = Window.partitionBy("placa")
#     df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
#     df2 = df2.join(df,['placa'],how='right')
#     df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit('acima_acel')).union(df_perigosa) 

def perigosa(df,col):
    global df_perigosa
    df = df.filter(F.col(col) == 1).select(F.col('placa'),F.col('tempo_da_simulacao'))
    window = Window.partitionBy("placa")
    df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
    df2 = df2.join(df,['placa'],how='right')
    df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit(col)).union(df_perigosa) 

In [13]:
p = [[key]+list(params[key].values()) for key in params.keys()]
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
df_old = ss.createDataFrame([], "placa: string, posicao: int, faixa: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int,tempo_inicio: int")

def pipeline(first=1,last=session.execute("select count(*) from simulacao").one()[0]):
    global p
    global df_old
    number = first
    while number < last:
        print(number)
        time0 = time.time()
        a, b = interval
        query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao BETWEEN {a} AND {b} ALLOW FILTERING;"
        r = list(session.execute(query))
        time1 = time.time()
        print('Leitura do banco tempo: ',time1-time0)
        time0 = time.time()
        if r != []:
            df = ss.createDataFrame(r)
            time1 = time.time()
            print('Criação do df tempo: ',time1-time0)
            time0 = time.time()
            df_old = processa_carro(df, df_old, 1, 0.5, p)
            time1 = time.time()
            print('Processa carros tempo: ',time1-time0)
            time0 = time.time()
            aplica_multa(df_old)
            time1 = time.time()
            print('Aplica multa tempo: ',time1-time0)
            time0 = time.time()
            perigosa(df_old,'acima_vel')
            perigosa(df_old,'acima_acel')
            perigosa(df_old,'troca_de_faixa')
            time1 = time.time()
            print('Perigosa tempo: ',time1-time0)
            time0 = time.time()
        
        number+=1
        # last = session.execute("select count(*) from simulacao").one()[0]

pipeline(168,171)

168


NameError: name 'interval' is not defined

In [ ]:
df_old.toPandas()

In [9]:
df_perigosa.toPandas()

,placa,perigosa_numero,tempo_da_simulacao,tipo
0,BRA2A34,2,170.0,troca_de_faixa
1,BRA2A34,2,170.0,acima_vel


In [10]:
df_multas.show()

+-------+------------+------------------+
|  placa|multa_numero|tempo_da_simulacao|
+-------+------------+------------------+
|BRA2A34|           1|             170.0|
+-------+------------+------------------+



In [62]:
fps = 30
vel_media = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}
n_vel_media = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}

tempo_medio = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}
n_tempo_medio = {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['vel_y'])
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean("vel_y"))
    mean_df = mean_df.withColumnRenamed("avg(vel_y)", "vel_y")
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    for row in mean_rows:
        print(row.asDict())
    mean_dict = {row.asDict()["rodovia"]: row.asDict()["vel_y"] for row in mean_rows}

    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("vel_y"))
    length_df = length_df.withColumnRenamed("count(vel_y)", "vel_y")
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row.asDict()["rodovia"]: row.asDict()["vel_y"] for row in length_rows}
    n_vel_media = {key: n_vel_media[key]+length_dict[key] for key in list(n_vel_media.keys())}
    vel_media = {key: atualiza_media(vel_media[key], n_vel_media[key], mean_dict[key], length_dict[key]) for key in list(vel_media.keys())}
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter((col("tempo_inicio").isNotNull()) & (col("tempo_final").isNotNull()))
    batch = batch.withColumn("tempo_cruzamento", col("tempo_final") - col("tempo_inicio"))
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean("tempo_cruzamento"))
    mean_df = mean_df.withColumnRenamed("avg(tempo_cruzamento)", "tempo_cruzamento")
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    mean_dict = {row.asDict()["rodovia"]: row.asDict()["tempo_cruzamento"] for row in mean_rows}

    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("tempo_cruzamento"))
    length_df = length_df.withColumnRenamed("count(tempo_cruzamento)", "tempo_cruzamento")
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row.asDict()["rodovia"]: row.asDict()["tempo_cruzamento"] for row in length_rows}
    n_tempo_medio = {key: n_tempo_medio[key]+length_dict[key] for key in list(n_tempo_medio.keys())}
    tempo_medio = {key: atualiza_media(tempo_medio[key], n_tempo_medio[key], mean_dict[key], length_dict[key]) for key in list(tempo_medio.keys())}
    pass

In [79]:
a, b = 0, 10000
query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
r = list(session.execute(query))
if r != []:
    df = ss.createDataFrame(r)

In [80]:
df.count()

12571

In [81]:
df.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               441|ARG3L45|475.0|            130.45|
| BR-116|               441|ARG7Z89|655.0|             748.0|
| BR-116|               441|BOL3R45|475.0|171.80000000000004|
| BR-116|               441|BOL7F89|745.0|             714.8|
| BR-116|               441|BOL9D01|835.0|193.99999999999966|
| BR-116|               441|BRA0K12|475.0|138.04999999999998|
| BR-116|               441|CHI0G12|385.0|             172.0|
| BR-116|               441|CHI6S78|385.0|163.35000000000002|
| BR-116|               441|COL1R23|835.0|             395.0|
| BR-116|               441|ECU8G90|565.0|116.85000000000001|
| BR-116|               441|GUY1X23|385.0| 299.8000000000002|
| BR-116|               441|PAR7P89|475.0|167.10000000000002|
| BR-116|               441|SUR0U12|655.0|             760.0|
| BR-116

In [82]:
p = [[key]+list(params[key].values()) for key in params.keys()]
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
df_old = ss.createDataFrame([], "placa: string, posicao: int, faixa: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int,tempo_inicio: int")

In [83]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col

df_cruzamento = ss.createDataFrame([], "rodovia: string, placa: string, tempo_inicio: bigint, tempo_final: bigint")

Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")

collision_tolerance = 1
collision_tolerance_quad = 0.5
windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))

#processa_velocidade_media(df)

df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)

window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")

# Add the lag column to the DataFrame
df = df.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lag(col("pos_x")).over(window_spec_rf)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lead(col("pos_x")).over(window_spec_rf))), 1).otherwise(0))

df = df.join(Velocidades_Maximas,on='rodovia',how='left')
df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')

df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))

#Calcula acima_vel

df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))

#Contador de trocas

df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))

#Multa

windowSpec = Window.partitionBy('rodovia',"placa").orderBy('tempo_da_simulacao')

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800))))
df = df.withColumn('tempo_inicio',F.col("tempo_da_simulacao")*when(((F.col('on_road') == 1) & ((lag('on_road').over(windowSpec) == 0) | (lag('on_road').over(windowSpec).isNull()))), 1).otherwise(None))
df = df.withColumn('tempo_final',F.col("tempo_da_simulacao")*(when((F.col('on_road') == 1) & ((lead('on_road').over(windowSpec) == 0) | (lead('on_road').over(windowSpec).isNull())), 1).otherwise(None)))

df_cruzamento_aux = df.withColumn("row",row_number().over(windowSpec)) \
                      .withColumn("t_inicio", first(col("tempo_inicio"), ignorenulls=True).over(windowSpec)) \
                      .withColumn("t_final", first(col("tempo_final"), ignorenulls=True).over(windowSpec)) \
                      .where(col("row")==1).select("rodovia", "placa",
                                               "t_inicio", "t_final").withColumnRenamed("rodovia", "rodovia2")

#join_condition = (df_cruzamento.rodovia == df_cruzamento_aux.rodovia2) & (df_cruzamento.placa == df_cruzamento_aux.placa2)
df_cruzamento = df_cruzamento.join(df_cruzamento_aux, on="placa", how="right")

df_cruzamento = df_cruzamento.withColumn("tempo_inicio", coalesce(df_cruzamento['tempo_inicio'], df_cruzamento['t_inicio']))
df_cruzamento = df_cruzamento.withColumn("tempo_final", coalesce(df_cruzamento['t_final'], df_cruzamento['tempo_final']))
df_cruzamento = df_cruzamento.drop("rodovia2", "t_inicio", "t_final")
df_cruzamento.filter(col("tempo_final").isNotNull()).show()
#processa_tempo_cruzamento(df_cruzamento)

#df_cruzamento = df_cruzamento
#F.lit(0))

# df = df.withColumn("time_on_road", col("on_road") * (1 + lag("time_on_road", 1).over(windowSpec)))

# windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao").rowsBetween(Window.unboundedPreceding, 0)

# # df = df.withColumn("cumulative_sum", F.sum(col("on_road")).over(windowSpec))

# # # windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

# df = df.withColumn("reset", when(col("time_on_road") != lag("time_on_road", 1).over(windowSpec), 1).otherwise(0))
# df = df.withColumn("group", F.sum("reset").over(windowSpec))
# df = df.withColumn("cumulative_sum", F.sum(col("placa")).over(windowSpec.partitionBy("group")))



+-------+-------+------------+-----------+
|  placa|rodovia|tempo_inicio|tempo_final|
+-------+-------+------------+-----------+
|PAR7P89|   null|         116|        116|
|PER5V67|   null|         245|        245|
|PER7T89|   null|         433|        433|
+-------+-------+------------+-----------+



In [84]:
dfp = df.toPandas()
dfp[dfp['placa'] == 'PAR7P89'].sort_values(by=['tempo_da_simulacao'], ascending=True)

,rodovia,tempo_da_simulacao,placa,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,...,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,tempo_inicio,tempo_final
8204,BR-116,116,PAR7P89,475.0,60.00,NaN,NaN,NaN,NaN,NaN,...,0,9,0.12,None,None,None,None,True,116.0,116.0
8205,BR-116,119,PAR7P89,745.0,863.00,60.00,116.0,267.666667,NaN,NaN,...,0,9,0.12,True,None,True,None,False,NaN,NaN
8206,BR-116,120,PAR7P89,745.0,856.00,863.00,119.0,-7.000000,267.666667,-274.666667,...,1,9,0.12,False,True,False,False,False,NaN,NaN
8207,BR-116,121,PAR7P89,745.0,849.05,856.00,120.0,-6.950000,-7.000000,0.050000,...,1,9,0.12,False,False,False,False,False,NaN,NaN
8208,BR-116,122,PAR7P89,745.0,842.15,849.05,121.0,-6.900000,-6.950000,0.050000,...,1,9,0.12,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8461,BR-116,1265,PAR7P89,565.0,445.50,97.05,1263.0,174.225000,-42.350000,108.287500,...,1,9,0.12,True,True,False,False,True,NaN,NaN
8462,BR-116,1286,PAR7P89,565.0,333.30,445.50,1265.0,-5.342857,174.225000,-8.550850,...,0,9,0.12,False,True,False,False,True,NaN,NaN
8463,BR-116,1297,PAR7P89,565.0,175.80,333.30,1286.0,-14.318182,-5.342857,-0.815939,...,0,9,0.12,True,True,False,True,True,NaN,NaN
8464,BR-116,1308,PAR7P89,565.0,271.35,175.80,1297.0,8.686364,-14.318182,2.091322,...,0,9,0.12,False,True,False,False,True,NaN,NaN
